In [81]:
import pandas as pd

In [82]:
#read in datasets and concat datasets

data7 = pd.read_csv('C:/Users/willy/Desktop/V_top_500.csv',header = None,encoding='ISO-8859-1',engine='python',error_bad_lines=False,keep_default_na=False,skip_blank_lines=True)
data = data7[:45032]
data[2] = data[2]+data[3]+data[4]+data[5]
df7=data.drop(labels=range(3,46),axis=1)

data1 = pd.read_csv('D:/datasets/1.csv',header = None)
data2 = pd.read_csv('D:/datasets/2.csv',header = None)
data3 = pd.read_csv('D:/datasets/3.csv',header = None)
data4 = pd.read_csv('D:/datasets/4.csv',header = None)
data5 = pd.read_csv('D:/datasets/5.csv',header = None)
data6 = pd.read_csv('D:/datasets/6.csv',header = None)
data8 = pd.read_csv('D:/datasets/8.csv',header = None)
data9 = pd.read_csv('D:/datasets/9.csv',header = None)
data10 = pd.read_csv('D:/datasets/10.csv',header = None)

dfs = [df7,data1,data2,data3,data4,data5,data6,data8,data9,data10]
df = pd.concat(dfs)
df = df.rename(columns={0:'Name',1:'Type',2:'Text'})
df = df.drop_duplicates()
df.dropna(axis=0, how='any', inplace=True)
df.index = range(len(df))

C:\Users\willy\AppData\Local\Temp/ipykernel_3144/1875190810.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data7 = pd.read_csv('C:/Users/willy/Desktop/V_top_500.csv',header = None,encoding='ISO-8859-1',engine='python',error_bad_lines=False,keep_default_na=False,skip_blank_lines=True)
C:\Users\willy\AppData\Local\Temp/ipykernel_3144/1875190810.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[2] = data[2]+data[3]+data[4]+data[5]


In [83]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import spacy
import emot
emot_obj = emot.core.emot()
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO

# Preprocess

In [117]:
def get_wordnet_pos(word):
    pack = nltk.pos_tag([word])
    tag = pack[0][1]
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV

    else:
        return None

def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, " "+UNICODE_EMOJI[emot]+" ")
    return text

def replace(new_sentence):
    replacement_patterns = [
            (r'won\'t', 'will not'),
            (r'can\'t', 'cannot'),
            (r'i\'m', 'i am'),
            (r'ain\'t', 'is not'),
            (r'(\w+)\'ll', '\g<1> will'),
            (r'(\w+)n\'t', '\g<1> not'),
            (r'(\w+)\'ve', '\g<1> have'),
            (r'(\w+)\'s', '\g<1> is'),
            (r'(\w+)\'re', '\g<1> are'),
            (r'(\w+)\'d', '\g<1> would')]
    patterns = [(re.compile(regex), repl) for (regex, repl) in replacement_patterns]

    for (pattern, repl) in patterns:
        (new_sentence, count) = re.subn(pattern, repl, new_sentence)
    return new_sentence

def remove_pun(new_sentence):
    punc = '~`!#$%^&*()_+-=|\';":/.,?><~·！@#￥%……&*（）——+-=“：’；、。，？》《{}'
    new_sentence = re.sub(r"[%s]+" %punc, "",new_sentence)
    new_sentence = new_sentence.replace('\n', ' ').replace('\r', '').replace('x88¥æª',' ').replace('\x88é',' ').replace('\x88',' ').replace('\x9a',' ').replace('\x87',' ')
    
    return new_sentence

def digit_remove(new_sentence):
    new_sentence =  " ".join([word for word in new_sentence.split() if not word.isdigit()])
    return new_sentence

def stopwords_remove(sentence):
    #stoplist = stopwords.words('english')
     
    with open('C:/Users/willy/Desktop/stopwords.txt') as file:
        stoplist = [stopword.replace('\n', '').lower() for stopword in file.readlines()]
    
    new_sentence = [word for word in sentence if word not in stoplist]
    return new_sentence

def lemmatize(sentence):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    new_sentence = [lemmatizer.lemmatize(word, get_wordnet_pos(word) or wordnet.NOUN) for word in sentence]
    return new_sentence


In [85]:
def preprocess_text(sentence):
    
    # 1.lowercase
    new_sentence = str(sentence).lower()
    
    # 2.emoji convert
    new_sentence = convert_emojis(new_sentence)
    
    # 3.Abbreviation expansion
    replacement_patterns = [
        (r'won\'t', 'will not'),
        (r'can\'t', 'cannot'),
        (r'i\'m', 'i am'),
        (r'ain\'t', 'is not'),
        (r'(\w+)\'ll', '\g<1> will'),
        (r'(\w+)n\'t', '\g<1> not'),
        (r'(\w+)\'ve', '\g<1> have'),
        (r'(\w+)\'s', '\g<1> is'),
        (r'(\w+)\'re', '\g<1> are'),
        (r'(\w+)\'d', '\g<1> would')]
    patterns = [(re.compile(regex), repl) for (regex, repl) in replacement_patterns]

    for (pattern, repl) in patterns:
        (new_sentence, count) = re.subn(pattern, repl, new_sentence)
    
    #4.Punctuation removal
    new_sentence = re.sub(',|!|\?|\"|<|>|\(|\)|\[|\]|\{|\}|@|#|\+|\=|\-|\_|~|\&|\*|\^|%|\||\$|/|`|\.|\'',
                          '', new_sentence,count=0, flags=0)
    #5.digits removal
    new_sentence =  " ".join([word for word in new_sentence.split() if not word.isdigit()])
    
    #6.Stopwords removal (including 16 types)
    with open('C:/Users/willy/Desktop/stopwords.txt') as file:
        stoplist = [stopword.replace('\n', '').lower() for stopword in file.readlines()]
    new_sentence = [word for word in new_sentence if word not in stoplist]
    
    #7.Lemmatization
    lemmatizer = nltk.stem.WordNetLemmatizer()

    new_sentence = [lemmatizer.lemmatize(word, get_wordnet_pos(word) or wordnet.NOUN) for word in new_sentence]
    

    
    #8.Sentence tokenization
    new_sentence = nltk.word_tokenize(new_sentence)
    

    
    return new_sentence

    

In [86]:
new_sentence_lower = df['Text'].apply(lambda text: text.lower())

In [87]:
new_sentence_noemoji = new_sentence_lower.apply(lambda text: convert_emojis(text))

In [88]:
new_sentence_replace = new_sentence_noemoji.apply(lambda text: replace(text))

In [108]:
new_sentence_nopun = new_sentence_replace.apply(lambda text: remove_pun(text))

In [118]:
new_sentence_nodigit = new_sentence_nopun.apply(lambda text: digit_remove(text))

In [120]:
new_sentence_tokenize = new_sentence_nodigit.apply(lambda text: nltk.word_tokenize(text))

In [121]:
new_sentence_stopwords = new_sentence_tokenize.apply(lambda text: stopwords_remove(text))

In [122]:
new_sentence_lemmatize = new_sentence_stopwords.apply(lambda text: lemmatize(text))

In [123]:
new_sentence_lemmatize

0        [reading, comment, heé, ¥æª, sexual, strength,...
1        [edit, not, answer, useless, decide, bring, vi...
2                                   [sasuke, limbic, calm]
3        [conflict, sasuke, type, frankly, distinguish,...
4        [not, understand, vote, sexual, main, motivati...
                               ...                        
83089    [type, alligns, value, cobra, kai, act, heavil...
83090                                      [person, annoy]
83091    [argument, not, facet, identify, feeling, peop...
83092    [dom, obsess, image, strive, emperor, sacrific...
83093    [personally, lot, dutiful, task, protect, fore...
Name: Text, Length: 83094, dtype: object

In [ ]:
df['Text'] = new_sentence_lemmatize
df.index = range(len(df))
df.to_csv('preprocess_data.csv')

In [146]:
df = pd.read_csv('preprocess_data.csv')

In [147]:
data = df.drop(index = df['Text'][df['Text'] == '[]'].index)

In [149]:
data.index = range(len(data))

In [150]:
data.to_csv('preprocess_data.csv')